In [68]:
# coding:utf-8

import pandas as pd

lgb80 = pd.read_csv('../datas/lgb60_month_1_yval_pred')

In [ ]:
# TODO 统计预测 topn 的 trade_cnt 分布，统计预测所有正例的 trade_cnt 分布
# TODO 预测的正例为什么评分不高(需要把评分提上来)，为什么有的负例的评分会很高
# TODO 统计有多少 trade 记录在 30 min 内有登陆行为
# TODO 换 ip,device 多的用户容易被盗号
# id:166968 从 5 月份总共有 41 次交易行为，但是模型能够学的具体哪一次是有风险
# 流程：
# 1. 获得用户 id 在 5、6 月份的交易次数
# 2. 获得 rowkey 在 6 月份的打分 y
# 3. 分析打分正确的正样本 id:166968,
# 4. 分析评分较高的负样本 id:86131，从 t6 中筛选评分较高的负样本 id，查看在 5、6 月份的交易情况，登陆情况
# 5. 分析评分较低的正样本，从 t6 中筛选评分较高的负样本 id，查看在 5、6 月份的交易情况，登陆情况

In [13]:
import numpy as np

In [21]:
trade = pd.read_csv('../datas/t_trade.csv')
trade.head()

,rowkey,time,id,is_risk
0,3,2015-01-01 00:00:41.0,27863,0
1,6,2015-01-01 00:00:46.0,115201,0
2,13,2015-01-01 00:01:02.0,143711,0
3,32,2015-01-01 00:03:00.0,172699,0
4,43,2015-01-01 00:03:52.0,53265,0


In [29]:
tmp = trade[trade['time']>='2015-05-01 00:00:00.0']
dt = tmp[['id','is_risk']].groupby('id',as_index=False)['is_risk'].agg({'sum':np.size})
dt = dt[dt['sum']>0]
dt.head()

,id,sum
0,23,1
1,25,2
2,28,1
3,37,1
4,41,4


In [69]:
t6 = trade[trade['time']>='2015-06-01 00:00:00.0']
t6 = t6.merge(lgb80[['rowkey','y']],on='rowkey',how='left')
t6.head()

,rowkey,time,id,is_risk,y
0,736366,2015-06-01 00:00:47.0,17722,0,0.009873
1,736401,2015-06-01 00:02:11.0,106642,0,0.004745
2,736412,2015-06-01 00:02:38.0,13303,0,0.003008
3,736434,2015-06-01 00:03:40.0,18949,0,0.004565
4,736459,2015-06-01 00:04:38.0,137277,0,0.002281


In [64]:
t6 = t6.merge(dt,on='id',how='left')
t6.head()

,rowkey,time,id,is_risk,y,sum
0,736366,2015-06-01 00:00:47.0,17722,0,0.012926,3
1,736401,2015-06-01 00:02:11.0,106642,0,0.004693,2
2,736412,2015-06-01 00:02:38.0,13303,0,0.003874,3
3,736434,2015-06-01 00:03:40.0,18949,0,0.003198,5
4,736459,2015-06-01 00:04:38.0,137277,0,0.002005,18


In [71]:
t6.loc[:50]

,rowkey,time,id,is_risk,y
0,757002,2015-06-03 16:52:56.0,34956,1,0.945296
1,758150,2015-06-03 20:50:04.0,84017,1,0.925860
2,759124,2015-06-03 23:03:21.0,108051,1,0.925467
3,785025,2015-06-07 21:41:44.0,162723,1,0.921441
4,785010,2015-06-07 21:38:35.0,162723,1,0.919112
5,784980,2015-06-07 21:30:43.0,162723,1,0.908553
6,756804,2015-06-03 16:13:48.0,40832,1,0.908429
7,769097,2015-06-05 17:28:42.0,35880,1,0.894828
8,769076,2015-06-05 17:26:10.0,35880,1,0.880388
9,769859,2015-06-05 20:10:50.0,38019,1,0.870464


In [70]:
t6 = t6.sort_values('y',ascending=False).reset_index(drop=True)
t6.loc[50:100]

,rowkey,time,id,is_risk,y
50,778801,2015-06-06 23:19:54.0,118520,1,0.555023
51,768826,2015-06-05 16:48:08.0,11337,1,0.548384
52,768880,2015-06-05 16:55:25.0,11337,1,0.547641
53,849865,2015-06-16 10:55:30.0,10055,0,0.545114
54,769053,2015-06-05 17:23:20.0,35880,1,0.542100
55,778777,2015-06-06 23:16:23.0,118520,1,0.541347
56,842639,2015-06-15 11:31:29.0,114455,0,0.537056
57,751796,2015-06-02 21:30:24.0,63336,0,0.533204
58,804489,2015-06-10 13:05:29.0,169467,0,0.533136
59,946043,2015-06-30 00:21:45.0,175546,1,0.532876


In [60]:
tmp[(tmp['id']==153164)&(tmp['time']>'2015-06-01 00:00:00.0')].sort_values('time').reset_index(drop=True)

,rowkey,time,id,is_risk
0,743926,2015-06-01 20:08:21.0,153164,0


In [46]:
t60 = t6[t6['is_risk']==0].sort_values('time').reset_index(drop=True)
t60.head(60)

,rowkey,time,id,is_risk,y,sum
0,736366,2015-06-01 00:00:47.0,17722,0,0.008874,3
1,736379,2015-06-01 00:01:15.0,145614,0,0.011722,7
2,736397,2015-06-01 00:01:59.0,143724,0,0.019595,5
3,736398,2015-06-01 00:02:01.0,118827,0,0.006016,1
4,736401,2015-06-01 00:02:11.0,106642,0,0.007875,2
5,736408,2015-06-01 00:02:31.0,165304,0,0.009923,5
6,736412,2015-06-01 00:02:38.0,13303,0,0.006082,3
7,736420,2015-06-01 00:02:54.0,33975,0,0.013066,1
8,736434,2015-06-01 00:03:40.0,18949,0,0.008349,5
9,736443,2015-06-01 00:04:11.0,135555,0,0.005270,4
